In [2]:
import sqlite3
import pandas as pd
conn = sqlite3.connect("vivino.db")
cursor = conn.cursor()


#### Question 1: Which 10 wines would you lik eto increase sales

##### Most expensive wines

In [3]:
# We want to highlight 10 wines to increase our sales. Which ones should we choose and why?
# FOR VINTAGE WINES: HIGH PRIC
cursor.execute('''SELECT DISTINCT vintages.name, vintages.ratings_average, vintages.ratings_count, vintages.price_euros 
               FROM vintages 
               WHERE vintages.ratings_count > 500
               ORDER BY  vintages.price_euros DESC, vintages.ratings_average DESC,vintages.ratings_count DESC
               LIMIT 10;''')
data = cursor.fetchall()
cols = ['vintage_name', 'rating_average', 'rating_count', 'price_vintage']
df1 = pd.DataFrame(data, columns=cols)
df1['revenue_vintages'] = df1['rating_count'] * df1['price_vintage']
df1

,vintage_name,rating_average,rating_count,price_vintage,revenue_vintages
0,Pétrus Pomerol 1982,4.7,1361,9180.00,12493980.00
1,Pétrus Pomerol 1990,4.8,1190,6300.00,7497000.00
2,Château Lafite Rothschild Pauillac (Premier Gr...,4.7,1273,5977.50,7609357.50
3,Masseto Toscana 2001,4.8,651,5231.25,3405543.75
4,Masseto Toscana 2006,4.8,797,4910.00,3913270.00
5,Masseto Toscana 2005,4.8,762,4697.50,3579495.00
6,Masseto Toscana 2010,4.7,1054,4697.50,4951165.00
7,Pétrus Pomerol 1979,4.7,883,4410.00,3894030.00
8,Masseto Toscana 2004,4.7,767,4163.75,3193596.25
9,Masseto Toscana 2008,4.7,764,4163.75,3181105.00


## TOTAL RATING COUNT OF EXCLUSIVE VINTAGES

In [4]:
import seaborn as sns
import plotly.express as px

df1_sorted = df1.sort_values(by='rating_count', ascending=False)
fig = px.bar(df1_sorted, x='vintage_name', y='rating_count', title='TOP 10: Total rating Count of exclusive Vintages')

fig.update_layout(
    yaxis_title='Rating Count',
    xaxis_title='Vintage Name',
    title_x=0.5
)

# fig.update_traces(text=df1['rating_count'], textposition='inside')
fig.update_traces(
    text=df1_sorted.apply(
        lambda row: f"{row['rating_count']}<br><span style='color: yellow;'>€ {row['price_vintage']:.2f} / unit</span>",
        axis=1
    ),
    textposition='inside'
)
# Show the plot
fig.show()

## TOTAL REVENUE OF EXCLUSIVE WINES

In [19]:
import plotly.express as px

df2_sorted = df1.sort_values(by='revenue_vintages', ascending=False)
fig = px.bar(df2_sorted, x='vintage_name', y='revenue_vintages', title='TOP 10: Highest revenue to date of highly rated exclusive Vintages')

fig.update_layout(
    yaxis_title='Total Revenue To Date (million €)',
    xaxis_title='Vintage Name',
    title_x=0.5
)

fig.update_traces(
    text=df2_sorted.apply(
        lambda row: f"€ {row['revenue_vintages']/1000000:.2f} m<br><span style='color: yellow;'>€ {row['price_vintage']:.2f} / unit</span>",
        axis=1
    ),
    textposition='inside'
)

# fig.update_traces(text=df2_sorted['revenue_vintages'].apply(lambda x: f"€ {x/1000000:.2f} m"), textposition='inside')

# Show the plot
fig.show()

In [21]:
## ADDING LOW END WINES 

cursor.execute('''SELECT DISTINCT vintages.name, vintages.ratings_average, vintages.ratings_count, vintages.price_euros 
               FROM vintages 
               WHERE vintages.ratings_count > 500
               ORDER BY vintages.ratings_count DESC, vintages.ratings_average DESC
               LIMIT 10;''')
data = cursor.fetchall()
cols = ['vintage_name', 'rating_average', 'rating_count', 'price_vintage']
df3 = pd.DataFrame(data, columns=cols)
df3['revenue_vintages'] = df3['rating_count'] * df3['price_vintage']

merged_df = pd.concat([df1,df3]) 
sorted_df = merged_df.sort_values(by='revenue_vintages', ascending=False)
sorted_df.head(10)


,vintage_name,rating_average,rating_count,price_vintage,revenue_vintages
0,Pétrus Pomerol 1982,4.7,1361,9180.0,12493980.0
4,Dom Pérignon Brut Champagne 2006,4.6,21121,440.0,9293240.0
2,Château Lafite Rothschild Pauillac (Premier Gr...,4.7,1273,5977.5,7609357.5
1,Pétrus Pomerol 1990,4.8,1190,6300.0,7497000.0
1,Krug Grande Cuvée,4.6,28513,245.0,6985685.0
8,Dom Pérignon Brut Champagne 2004,4.6,14650,460.0,6739000.0
9,Antinori Tignanello 2016,4.6,13709,420.0,5757780.0
6,Masseto Toscana 2010,4.7,1054,4697.5,4951165.0
4,Masseto Toscana 2006,4.8,797,4910.0,3913270.0
7,Pétrus Pomerol 1979,4.7,883,4410.0,3894030.0


## HIGHEST REVENUE OF HIGHEST RATED WINES

In [22]:
import plotly.express as px

sorted_df = sorted_df.head(10)
fig = px.bar(sorted_df, x='vintage_name', y='revenue_vintages', title='TOP 10: Highest revenue to date of highest rated wines (total ratings > 500)')

fig.update_layout(
    yaxis_title='Total revenue to date (million €)',
    xaxis_title='Vintage Name',
    title_x=0.5
)

fig.update_traces(
    text=sorted_df.apply(
        lambda row: f"€ {row['revenue_vintages']/1000000:.2f} m<br><span style='color: yellow;'>€ {row['price_vintage']:.2f} / unit</span>",
        axis=1
    ),
    textposition='inside'
)

# Show the plot
fig.show()

#### Question 2: What country should we focus on?

##### Answer: FOCUS ON FRANCE:given the highest revnue per country

In [23]:
cursor.execute('''SELECT DISTINCT countries.name, vintages.name, vintages.ratings_average, vintages.ratings_count, vintages.price_euros 
               FROM wines
               JOIN vintages ON wine_id == wines.id
               JOIN regions ON region_id == regions.id
               JOIN countries ON regions.country_code == countries.code 
               WHERE vintages.ratings_count > 500
               ORDER BY  vintages.price_euros DESC, vintages.ratings_average DESC,vintages.ratings_count DESC
               LIMIT 10;''')
data = cursor.fetchall()
cols = ['country', 'vintage_name', 'rating_average', 'rating_count', 'price_vintage']
df3 = pd.DataFrame(data, columns=cols)
df3['revenue_vintages'] = df3['rating_count'] * df3['price_vintage']
df3 

,country,vintage_name,rating_average,rating_count,price_vintage,revenue_vintages
0,France,Pétrus Pomerol 1982,4.7,1361,9180.00,12493980.00
1,France,Pétrus Pomerol 1990,4.8,1190,6300.00,7497000.00
2,France,Château Lafite Rothschild Pauillac (Premier Gr...,4.7,1273,5977.50,7609357.50
3,Italie,Masseto Toscana 2001,4.8,651,5231.25,3405543.75
4,Italie,Masseto Toscana 2006,4.8,797,4910.00,3913270.00
5,Italie,Masseto Toscana 2005,4.8,762,4697.50,3579495.00
6,Italie,Masseto Toscana 2010,4.7,1054,4697.50,4951165.00
7,France,Pétrus Pomerol 1979,4.7,883,4410.00,3894030.00
8,Italie,Masseto Toscana 2004,4.7,767,4163.75,3193596.25
9,Italie,Masseto Toscana 2008,4.7,764,4163.75,3181105.00


In [24]:
df3_sorted = df3.sort_values(by='revenue_vintages', ascending=False)
fig = px.bar(df3_sorted, x='vintage_name', y='revenue_vintages', color='country', title='TOP 10: Highest revenue to date of highest rated wines (total ratings > 500)')

fig.update_layout(
    yaxis_title='Total Revenue To Date (million €)',
    xaxis_title='Vintage Name',
    title_x=0.5 # Adjust bottom margin to make space for text annotations
)

# fig.update_traces(text=df3_sorted['revenue_vintages'].apply(lambda x: f"€ {x/1000000:.2f} m"), textposition='inside')
for trace in fig.data:
    text_positions = []
    for i, (y, country) in enumerate(zip(trace.y, df3_sorted['country'])):
        if country == trace.name:
            text_positions.append(y)  # Adjust the value for proper placement
        else:
            text_positions.append(y)  # Adjust the value for proper placement
    trace.text = [f'€ {x/1000000:.2f} m' for x in text_positions]
    trace.textposition = 'inside'

fig.show()


In [11]:
# SKIP QUESTION 3

#### Question 4-5: specific combination of tastes

In [12]:
test = '''  
SELECT 
    wines.name,
    GROUP_CONCAT(DISTINCT keywords.name) AS mixed_tastes,
    keywords_wine.group_name,
    COUNT(DISTINCT keywords.name) as count_of_tastes
FROM 
    wines
JOIN 
    regions ON wines.region_id = regions.id
JOIN 
    countries ON regions.country_code = countries.code
JOIN 
    keywords_wine ON wines.id = keywords_wine.wine_id
JOIN 
    keywords ON keywords_wine.keyword_id = keywords.id
WHERE 
    keywords.name IN ('coffee', 'toast', 'green apple', 'cream', 'citrus')
AND 
    keywords_wine.count > 9
AND 
    keywords_wine.keyword_type = 'primary'
GROUP BY
    wines.name
HAVING 
    COUNT(DISTINCT keywords.name) >= 2

ORDER BY count_of_tastes DESC;
'''
cursor.execute(test)
data = cursor.fetchall()

columns = ['wine_name', 'mixed_tastes', 'taste_group', 'count_of_tastes']
df_3 = pd.DataFrame(data, columns=columns)
# pd.set_option('display.max_rows', None)
df_3


,wine_name,mixed_tastes,taste_group,count_of_tastes
0,Vintage,"toast,coffee,cream,green apple,citrus",non_oak,5
1,Trebbiano d'Abruzzo,"coffee,green apple,citrus,toast,cream",oak,5
2,Sir Winston Churchill Brut Champagne,"toast,coffee,cream,green apple,citrus",non_oak,5
3,Sauternes (Premier Grand Cru Classé),"green apple,coffee,toast,citrus,cream",tree_fruit,5
4,Sauternes,"green apple,coffee,toast,citrus,cream",tree_fruit,5
...,...,...,...,...
226,Barolo Cascina Francia,"coffee,cream",oak,2
227,Barbaresco,"coffee,citrus",oak,2
228,Aszú 6 puttonyos,"citrus,cream",citrus_fruit,2
229,Amour de Deutz Millesimé Rosé Brut Champagne,"cream,citrus",microbio,2


#### Question 4: 
- Find the top 3 most common grape all over the world and for each grape
- give us the 5 best rated wines. - IGNORE CANNOT BE ANSWERED

In [14]:
# TOP 3 MOST COMMON GRAPES
cursor.execute( """SELECT most_used_grapes_per_country.grape_id, grapes.name, most_used_grapes_per_country.wines_count, COUNT(most_used_grapes_per_country.grape_id) AS grapeCount
             FROM most_used_grapes_per_country
             JOIN grapes ON most_used_grapes_per_country.grape_id = grapes.id
             GROUP BY grape_id
             ORDER BY grapeCount DESC, wines_count DESC
             LIMIT 5
             """ )

cursor.fetchall() 

[(2, 'Cabernet Sauvignon', 801751, 12),
 (10, 'Merlot', 566719, 11),
 (5, 'Chardonnay', 604208, 6),
 (14, 'Pinot Noir', 572334, 3),
 (1, 'Shiraz/Syrah', 551112, 3)]

#### REASONING WHY WE CANNOT LINK THE WINES TO THE GRAPES


- There is a clear link between the wines and the countries 

- wines.region_id --> regions.id > regions.country_code --> countries.code > most_used_grapes_per_country.country_code > grapes.name

Unfortunately we cannot link the wines to grapes name because the link ends at the "most_used_grapes_per_country" node. This only shows the most_used_grapes per country (every wine from that country will have these same grapes listed even though it would not necessarily mean that one of these grapes is used to produce the wine)

#### Question 5: We would to give create a country leaderboard, give us a visual that shows the average wine rating for each country. Do the same for the vintages.

In [18]:
cursor.execute( """
SELECT 
    countries.name, 
    ROUND(AVG( vintages.ratings_average),2) AS average_rating_vintage,
    ROUND(AVG(wines.ratings_average),2) AS average_rating_wine
    
FROM 
    vintages
    JOIN wines ON vintages.wine_id = wines.id
    JOIN regions ON wines.region_id = regions.id
    JOIN countries ON regions.country_code = countries.code

GROUP BY countries.name
ORDER BY average_rating_vintage DESC, average_rating_wine DESC;


    """)
data = cursor.fetchall()
cols = ['countries', 'rating_average_vintage','rating_average_wine']
df_1 = pd.DataFrame(data, columns=cols)
df_1

,countries,rating_average_vintage,rating_average_wine
0,Roumanie,4.50,4.40
1,Croatie,4.40,4.30
2,Argentine,4.32,4.43
3,Chili,4.12,4.47
4,Portugal,3.93,4.43
5,Moldavie,3.87,4.47
6,États-Unis,3.70,4.51
7,Afrique du Sud,3.66,4.47
8,Italie,3.63,4.48
9,Espagne,3.38,4.47
